In [46]:
import re
import os
import json
import pandas as pd

In [47]:
toc_file = "toc.txt"
chapter_data = "booktext1.txt"

In [48]:
toc_array = []
file = open(toc_file)
for line in file:
    line = line.strip()
    if line != "\n" and re.search('[a-zA-Z0-9]', line):
        toc_array.append(line)
file.close()

In [49]:
book_array = []
file = open(chapter_data)
for line in file:
    line = line.strip()
    if line != "\n" and re.search('[a-zA-Z0-9]', line):
        book_array.append(line)
file.close()

In [50]:
def data_separation(line):
    line = line.strip()
    pos = line.find(" ")
    index_num = line[:pos]
    title = line[pos+1:]
    return (index_num, title)

def create_file(line):
    line = line.replace(" ", "_")
    file_name = "book_data/" + line + ".txt"
    file = open(file_name, "w+")
    file.close()
    return file_name

def match_line(line1, line2):
    line1 = line1.lower()
    line2 = line2.lower()
    index1, title = data_separation(line1)
    if index1 in line2 or index1+"." in line2:
        if title in line2: return 1
        else: return 0
    else:
        return 0


In [51]:
toc_len = len(toc_array)
book_len = len(book_array)

i = 0

current_line = toc_array[i]
next_line = toc_array[i + 1]

file_name= create_file(current_line)
current_file = open(file_name, "a")

for j in range(book_len):
    line = book_array[j]
    if match_line(next_line, line):
        if i < toc_len - 2:
            i += 1
            current_line = toc_array[i]
            next_line = toc_array[i + 1]
        else:
            current_line = toc_array[toc_len - 1]
        current_file.close()
        file_name = create_file(current_line)
        current_file = open(file_name, "a")
    else:
        line += "\n"
        current_file.write(line)
        
current_file.close()


In [69]:
book_data = {}
i = 0

for topic in toc_array:
    index_num, title = data_separation(topic)
    topic = topic.replace(" ", "_")
    file_name = "book_data/" + topic + ".txt"
    file = open(file_name, "r")
    content = file.read()
    file.close()
    data = {
        'Section': index_num,
        'Title': title,
        'Content': content
    }
    book_data[i] = data
    i += 1

## Clean the extracted data

In [109]:
def add_line(content, line, state):
    line = line.strip()
    if state == True:
        content += " " + line
    else: content += line
    return content

def incomplete_words(content):
    content_array = content.split("\n")
    state = True
    new_content = ""
    for i in range(len(content_array)-1):
        if content_array[i][-1] == "-":
            new_content = add_line(new_content, content_array[i][:-1], state)
            state = False
        else:
            new_content = add_line(new_content, content_array[i], state)
            state = True
    new_content = new_content.strip()
    return new_content

def handle_special_char(content):
    new_content = ""
    punctuations = "!\"#$%&()*+-.,:;<=>?@[\]^_`{|}~\n"
    for ch in content:
        if ch == "'":
            ch = ""
        if ch in punctuations:
            ch = " "
        if ord(ch) < 97 or ord(ch) > 122:
            ch = " "
        new_content += ch
    return new_content

def handle_words(content):
    words = content.split(" ")
    new_list = [word for word in words if len(word) > 2]
    content = " ".join(new_list)
    return content


def clean_content(content):
    content = content.lower()
    content = incomplete_words(content)
    content = handle_special_char(content)
    content = re.sub("\s\s+" , " ", content)
    content = handle_words(content)
    return content


In [110]:
for i in range(len(book_data)):
    book_data[i]["Content"] = clean_content(book_data[i]["Content"])

In [112]:
for i in range(10):
    print(book_data[i]["Content"])
    print("\n\n")

introduction the problem searching for patterns data fundamental one and has long and successful history for instance the extensive astronomical observations tycho brahe the century allowed johannes kepler discover the empirical laws planetary motion which turn provided springboard for the development classical mechanics similarly the discovery regularities atomic spectra played key role the development and verification quantum physics the early twentieth century the field pattern recognition concerned with the automatic discovery regularities data through the use computer algorithms and with the use these regularities take actions such classifying the data into different categories consider the example recognizing handwritten digits illustrated figure each digit corresponds pixel image and can represented vector comprising real numbers the goal build machine that will take such vector input and that will produce the identity the digit the output this nontrivial problem due the wide va

## Saving data in JSON Format

In [113]:
with open('book_data.json', 'w') as file:
    json.dump(book_data, file)

## Saving data in CSV format

In [114]:
df = pd.DataFrame(columns=['Section', 'Title', 'Content'])

for i in range(len(book_data)):
    df = df.append(book_data[i], ignore_index=True)

df.to_csv("book_data.csv")